In [ ]:
from pymc.math import exp
import numpy as np

data = np.loadtxt("gastric.txt")
x = data[:, 0].copy()
censored_vals = data[:, 1].copy()
y = data[:, 2].copy()

# we need to separate the observed values and the censored values
observed_mask = censored_vals == 0

censored = censored_vals[~observed_mask]
y_uncensored = y[observed_mask]
x_censored = x[~observed_mask]
x_uncensored = x[observed_mask]

In [ ]:
with pm.Model() as ex2:
    # Everything that happens in here is one iteration
    beta0 = pm.Normal("beta0", tau=0.001)
    beta1 = pm.Normal("beta1", tau=0.001)
    alpha = pm.Exponential("alpha", 3)

    lambda_censored = exp(beta0 + beta1 * x_censored)
    beta_censored = lambda_censored ** (-1 / alpha)

    lambda_uncensored = exp(beta0 + beta1 * x_uncensored)
    beta_uncensored = lambda_uncensored ** (-1 / alpha)

    impute_censored = pm.Bound(
        "impute_censored",
        pm.Weibull.dist(alpha=alpha, beta=beta_censored),
        lower=censored,
        shape=censored.shape[0],
    )

    likelihood = pm.Weibull(
        "likelihood",
        alpha=alpha,
        beta=beta_uncensored,
        observed=y_uncensored,
        shape=y_uncensored.shape[0],
    )

    trace_2 = pm.sample(10000, tune=2000, cores=4, init="auto", target_accept=0.9)

az.summary(trace_2)